### Problem 2 

Steps:
1) Import the Honeywell data set.
2) Using the double exponential smoothing technique calculate the results at different levels of alpha
3) Support the assertion by calculating the mean square error

Import Libraries

In [23]:
import warnings
import itertools
import numpy as np
import random
import pandas as pd
from scipy.stats import uniform
import math
import seaborn as sns
from matplotlib import pyplot as plt
import scipy 
from scipy.stats import chisquare
from scipy.stats import norm, chi2, expon

Read the honeywell file and write it to a dataframe

In [24]:
df = pd.read_excel("Honeywell.xlsx")
close = df['Close']
close.shape[0]
alpha = 0.75

In [25]:
df.head()

,Period,Date,Close,Volume
0,1,2017-10-16,143.750000,2001900
1,2,2017-10-17,143.429993,2435600
2,3,2017-10-18,143.440002,1734100
3,4,2017-10-19,143.619995,2623100
4,5,2017-10-20,145.350006,3369500


### Double Exponential Smoothing Formula : 

Double exponential smoothing employs a level component and a trend component at each period. Double exponential smoothing uses two weights, (also called smoothing parameters), to update the components at each period. The double exponential smoothing equations are as follows:

Level lx = ℓxTxy^x+1=αyx+(1−α)(ℓx−1+Tx−1)

Trend Tx = β(ℓx−ℓx−1)+(1−β)Tx−1

1step forecast = ℓx+Tx 

If the first observation is numbered one, then level and trend estimates at time zero must be initialized in order to proceed. The initialization method used to determine how the smoothed values are obtained in one of two ways: with optimal weights or with specified weights.

Notation

Lt - level at time t

α  - weight for the level

Tt - trend at time t

γ  - weight for the trend

Yt - data value at time t fitted value, or one-step-ahead forecast, at time t

In [26]:
def double_exponential_smoothing(close, alpha, beta):
    """given a close and alpha, return close of expoentially smoothed points"""
    
    n_record = close.shape[0]
    results = np.zeros_like(close)
    
    level = close[0]
    results[0] = close[0]
    trend = close[1] - close[0]
    
    for t in range(1, n_record):
        if t >= n_record:
            # forecasting new points
            value = results[t]
        else:
            value = close[t]
        
        previous_level = level
        level = alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - previous_level) + (1 - beta) * trend 
        results[t] = level + trend

    return results

In [27]:
k = double_exponential_smoothing(close.values, alpha, 0.45)
len(k)

124

In [28]:
df['DEsmoothing15'] = double_exponential_smoothing(close.values, alpha, 0.15)
df['DEsmoothing25'] = double_exponential_smoothing(close.values, alpha, 0.25)
df['DEsmoothing45'] = double_exponential_smoothing(close.values, alpha, 0.45)
df['DEsmoothing85'] = double_exponential_smoothing(close.values, alpha, 0.85)

In [29]:
df.head()

,Period,Date,Close,Volume,DEsmoothing15,DEsmoothing25,DEsmoothing45,DEsmoothing85
0,1,2017-10-16,143.750000,2001900,143.750000,143.750000,143.750000,143.750000
1,2,2017-10-17,143.429993,2435600,143.109986,143.109986,143.109986,143.109986
2,3,2017-10-18,143.440002,1734100,143.074618,143.099369,143.148871,143.247876
3,4,2017-10-19,143.619995,2623100,143.262125,143.329327,143.452592,143.654569
4,5,2017-10-20,145.350006,3369500,144.841397,145.063202,145.466407,146.134592


Calculate the forecasting value for row 124 (2018-04-16) as below

In [30]:
def double_exp(beta):
    results = np.zeros_like(close)
    level = close[122]
    results[122] = close[122]
    trend = close[123] - close[122]

    previous_level = level
    level = alpha * results[122] + (1 - alpha) * (level + trend)
    trend = beta * (level - previous_level) + (1 - beta) * trend 
    results[123] = level + trend

    return (results[123])

In [31]:
print(double_exp(0.15))
print(double_exp(0.25))
print(double_exp(0.45))
print(double_exp(0.85))

146.071869175
146.09811962499998
146.150620525
146.255622325


Append the above values into the dataframe row as below

In [36]:
# Pass a series in append() to append a row in dataframe  
df = df.append(pd.Series([125,'2018-04-14 00:00:00',0,0,146.07,146.09,146.15,146.25], index=df.columns ), ignore_index=True)

In [40]:
df15 = df['DEsmoothing15']
df25 = df['DEsmoothing25']
df45 = df['DEsmoothing45']
df85 = df['DEsmoothing85']

close = df['Close']

In [38]:
mse = [(close[i] - df15[i])**2 for i in range(124)]
k = np.mean(mse)
print("MSE at alpha 0.75 and Beta 0.15 is %s" % k)

mse1 = [(close[i] - df25[i])**2 for i in range(124)]
k1 = np.mean(mse1)
print("MSE at alpha 0.75 and Beta 0.25 is %s" % k1)

mse2 = [(close[i] - df45[i])**2 for i in range(124)]
k2 = np.mean(mse2)
print("MSE at alpha 0.75 and Beta 0.45 is %s" % k2)

mse3 = [(close[i] - df85[i])**2 for i in range(124)]
k3 = np.mean(mse3)
print("MSE at alpha 0.75 and Beta 0.85 is %s" % k3)

print("The Best MSE at alpha 0.75 is", min(k,k1,k2,k3), "beta value for best MSE is 0.15" )

MSE at alpha 0.75 and Beta 0.15 is 0.2807363737170363
MSE at alpha 0.75 and Beta 0.25 is 0.382769771616424
MSE at alpha 0.75 and Beta 0.45 is 0.6038156869133915
MSE at alpha 0.75 and Beta 0.85 is 1.092170122750298
The Best MSE at alpha 0.75 is 0.2807363737170363 beta value for best MSE is 0.15


In [39]:
df.tail(n=10)

,Period,Date,Close,Volume,DEsmoothing15,DEsmoothing25,DEsmoothing45,DEsmoothing85
116,117,2018-04-04 00:00:00,144.839996,2769000,144.206333,144.440185,144.982767,145.619593
117,118,2018-04-05 00:00:00,146.399994,2239600,145.854123,146.244915,146.937956,147.634037
118,119,2018-04-06 00:00:00,142.740005,2867900,143.170741,143.293935,143.264953,142.272710
119,120,2018-04-09 00:00:00,142.830002,1952400,142.529060,142.536701,142.267404,141.355150
120,121,2018-04-10 00:00:00,145.630005,2880400,144.817498,145.027389,145.252897,145.950982
121,122,2018-04-11 00:00:00,144.279999,1953100,144.316634,144.497421,144.658413,145.022184
122,123,2018-04-12 00:00:00,146.470001,2327600,146.076174,146.377289,146.763704,147.355469
123,124,2018-04-13 00:00:00,146.119995,1588500,146.258484,146.536509,146.810271,146.888671
124,125,2018-04-14 00:00:00,0.000000,0,146.070000,146.090000,146.150000,146.250000
125,125,2018-04-14 00:00:00,0.000000,0,146.070000,146.090000,146.150000,146.250000


### Problem 2 : Conclusion

1) The Best Mean Square Error at alpha 0.75 is 0.28 at beta value for best MSE is 0.15.Hence the forecasting closing value for stock price at honeywell on 2018-04-16 is 146.07
